# Model 시각화 및 학습 현황 확인 방법 정리.
https://gaussian37.github.io/dl-pytorch-observe/

## 1. torchsummary


| \ |torch 제공 ResNet|Custom ResNet|
|:------|---:|---:|
|Total params|25,557,032|23,528,586|
|Forward/backward pass size (MB)|351.15|251.68|
|Params size (MB)|97.49|89.75|
|Estimated Total Size (MB)|449.33|342.12|

차이 안날 줄 알았는데 조~금 차이나네

In [1]:
from torchvision import models
from torchsummary import summary

In [7]:
device='cuda'

mymodel = models.resnet50()
mymodel.to(device)
summary(mymodel, (3, 244, 244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]           4,096
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
           Conv2d-11          [-1, 256, 61, 61]          16,384
      BatchNorm2d-12          [-1, 256, 61, 61]             512
           Conv2d-13          [-1, 256, 61, 61]          16,384
      BatchNorm2d-14          [-1, 256,

In [9]:
from resnet import ResNet50

In [10]:
device='cuda'

mymodel = ResNet50()
mymodel.to(device)
summary(mymodel, (3, 244, 244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,472
       BatchNorm2d-2         [-1, 64, 122, 122]             128
         MaxPool2d-3           [-1, 64, 61, 61]               0
            Conv2d-4           [-1, 64, 61, 61]           4,096
       BatchNorm2d-5           [-1, 64, 61, 61]             128
            Conv2d-6           [-1, 64, 61, 61]          36,864
       BatchNorm2d-7           [-1, 64, 61, 61]             128
            Conv2d-8          [-1, 256, 61, 61]          16,384
       BatchNorm2d-9          [-1, 256, 61, 61]             512
           Conv2d-10          [-1, 256, 61, 61]          16,384
      BatchNorm2d-11          [-1, 256, 61, 61]             512
       BottleNeck-12          [-1, 256, 61, 61]               0
           Conv2d-13           [-1, 64, 61, 61]          16,384
      BatchNorm2d-14           [-1, 64,

## 2. torchinfo

안타깝게도 앞에서 설명한 torchsummary의 업데이트가 되지 않는 반면 새로운 모델 정보 요약 라이브러리인 torchinfo가 많이 사용되고 있습니다.   
-torchinfo는 기존의 torchsummary와 사용 방법은 거의 같습니다.   
더구나 기존의 torchsummary에서 LSTM과 같은 RNN 계열의 Summary 시 일부 오류가 났던 문제와 layer 분류를 좀 더 계층적으로 상세히 해준다는 점 등의 개선이 있어서 torchsummary 대신 torchinfo를 사용하는 것을 추천 드립니다.   
사용 방법의 일부 차이점은 torchinfo에서는 (batch, channel, height, width)와 같은 형태로 데이터를 입력 받습니다.   
이는 torchsummary 에서 batch를 사용하지 않고 (channel, height, width)를 사용하는 것과 차이점입니다.   
실제 사용하는 입장에서는 batch를 고려하여 모델을 설계하는 것이 더 현실적이기 때문에 torchinfo를 사용하는 것이 더 좋다고 생각이 듭니다.  
아래는 VGG16에 (3, 224, 224) 크기의 이미지를 250개의 batch를 통하여 feedforward 하였을 때의 상태를 요약해 주는 예제 코드 입니다.   

In [15]:
!pip install torchinfo

In [20]:
import torch
from torchvision import models
from torchinfo import summary
'''
device = "cuda"
inp = torch.rand(1, 3, 224, 224).to(device)
resnet = models.resnet50().to(device)

summary(resnet, inp) # 250 = batch
'''

resnet = models.resnet50()
summary(resnet, (1,3,224,224))


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   --                        --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           4,096
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│ 

마지막의 Estimated Total Size를 참조하면 실제 학습할 때 사용할 GPU 양을 계산할 수 있습니다.   
이를 통해 현재 GPU 자원을 기준으로 적당한 batch 크기를 계산할 수 있습니다.    
Total mult-adds (G): 4.09 이게 flops다

-----
# FLOPs compute
pthflops라는 라이브러리 다운로드받고 flops계산했다.

In [1]:
!pip install pthflops

In [11]:
import torch
from torchvision.models import resnet18

from pthflops import count_ops

# Create a network and a corresponding input
device = 'cuda:0'
model = resnet18().to(device)
inp = torch.rand(64,3,224,224).to(device)

# Count the number of FLOPs
count_ops(model, inp)

Operation               OPS          
----------------------  -----------  
conv1                   7552892928   
bn1                     102760448    
relu                    102760448    
maxpool                 102760448    
layer1_0_conv1          7398752256   
layer1_0_bn1            25690112     
layer1_0_relu           25690112     
layer1_0_conv2          7398752256   
layer1_0_bn2            25690112     
add                     25690112     
layer1_0_relu_1         25690112     
layer1_1_conv1          7398752256   
layer1_1_bn1            25690112     
layer1_1_relu           25690112     
layer1_1_conv2          7398752256   
layer1_1_bn2            25690112     
add_1                   25690112     
layer1_1_relu_1         25690112     
layer2_0_conv1          3699376128   
layer2_0_bn1            12845056     
layer2_0_relu           12845056     
layer2_0_conv2          7398752256   
layer2_0_bn2            12845056     
layer2_0_downsample_0   411041792    
layer2_0_dow

(116914750440,
 [['conv1', 7552892928],
  ['bn1', 102760448],
  ['relu', 102760448],
  ['maxpool', 102760448],
  ['layer1_0_conv1', 7398752256],
  ['layer1_0_bn1', 25690112],
  ['layer1_0_relu', 25690112],
  ['layer1_0_conv2', 7398752256],
  ['layer1_0_bn2', 25690112],
  ['add', 25690112],
  ['layer1_0_relu_1', 25690112],
  ['layer1_1_conv1', 7398752256],
  ['layer1_1_bn1', 25690112],
  ['layer1_1_relu', 25690112],
  ['layer1_1_conv2', 7398752256],
  ['layer1_1_bn2', 25690112],
  ['add_1', 25690112],
  ['layer1_1_relu_1', 25690112],
  ['layer2_0_conv1', 3699376128],
  ['layer2_0_bn1', 12845056],
  ['layer2_0_relu', 12845056],
  ['layer2_0_conv2', 7398752256],
  ['layer2_0_bn2', 12845056],
  ['layer2_0_downsample_0', 411041792],
  ['layer2_0_downsample_1', 12845056],
  ['add_2', 12845056],
  ['layer2_0_relu_1', 12845056],
  ['layer2_1_conv1', 7398752256],
  ['layer2_1_bn1', 12845056],
  ['layer2_1_relu', 12845056],
  ['layer2_1_conv2', 7398752256],
  ['layer2_1_bn2', 12845056],
  ['add_

In [14]:
import torch
from torchvision.models import resnet50, mobilenet_v2
#from resnet import ResNet50

from pthflops import count_ops

# Create a network and a corresponding input
device = 'cuda:1'
model = resnet50().to(device) 
#MobileNetV2 = mobilenet_v2().to(device)
inp = torch.rand(1,3,224,224).to(device)

# Count the number of FLOPs
count_ops(model, inp)
#count_ops(MobileNetV2, inp)

'''
custom resnet50 = 3.89 GFLOPs
torchvision resnet50 = 4.14 GFLOPs
'''

Operation               OPS         
----------------------  ----------  
conv1                   118013952   
bn1                     1605632     
relu                    1605632     
maxpool                 1605632     
layer1_0_conv1          12845056    
layer1_0_bn1            401408      
layer1_0_relu           401408      
layer1_0_conv2          115605504   
layer1_0_bn2            401408      
layer1_0_relu_1         401408      
layer1_0_conv3          51380224    
layer1_0_bn3            1605632     
layer1_0_downsample_0   51380224    
layer1_0_downsample_1   1605632     
add                     1605632     
layer1_0_relu_2         1605632     
layer1_1_conv1          51380224    
layer1_1_bn1            401408      
layer1_1_relu           401408      
layer1_1_conv2          115605504   
layer1_1_bn2            401408      
layer1_1_relu_1         401408      
layer1_1_conv3          51380224    
layer1_1_bn3            1605632     
add_1                   1605632     
l

'\ncustom resnet50 = 3.89 GFLOPs\ntorchvision resnet50 = 4.14 GFLOPs\n'